In [3]:
from pyspark.sql import SparkSession
import pyspark.sql.functions as F 
master_url = "spark://localhost:7077"
driver_host_ip = "host.docker.internal"

In [5]:
spark = (
    SparkSession.builder.appName("LocalDriverToDockerCluster")
    .master(master_url)
    .config("spark.driver.host", driver_host_ip)
    .config("spark.executor.memory", "1G")
    .config("spark.executor.cores", "2")
    .getOrCreate()
)

PySparkRuntimeError: [JAVA_GATEWAY_EXITED] Java gateway process exited before sending its port number.